In [1]:
from pyspark.sql import SparkSession
import datetime,os
        # .config("spark.driver.port","4040") \
        # .config("spark.blockManager.port","4041") \
spark =  SparkSession.builder \
        .appName("Upserts Operation")\
        .master("local[*]") \
        .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
        .config("spark.driver.bindAddress","127.0.0.1") \
        .config("spark.executor.memory","4g") \
        .config("spark.driver.memory","8g") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .getOrCreate()
# spark.sparkContext.setLogLevel("ERROR")
spark.sparkContext.setLogLevel("WARN")


print(f'spark session created {spark} at {datetime.datetime.now()}')

# print(jars)
# spark.stop()

:: loading settings :: url = jar:file:/home/jarvis/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jarvis/.ivy2/cache
The jars for the packages stored in: /home/jarvis/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fd9bb074-2dae-4422-aec8-77720cc8a332;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 200ms :: artifacts dl 7ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |

spark session created <pyspark.sql.session.SparkSession object at 0x7b263f5f38c0> at 2025-06-14 23:43:44.614718


In [16]:
from pyspark.sql.functions import current_timestamp,lit
from delta.tables import DeltaTable

read_options={
    "header":True,
    "inferSchema":True
}
'''Read from source csv file'''
customer_master_src = spark.read.options(**read_options).csv('../datasets/customer_master.csv')
customer_master_src = customer_master_src.withColumn("created_at",current_timestamp())\
                .withColumn("updated_at",current_timestamp())\
                .withColumn("current_flag",lit("Y"))
                
# customer_master_src.show(truncate=False)
'''Write csv into deltalake'''
try:
    customer_master_src.coalesce(1).write.format("delta").mode("overwrite").save("../DeltaLake/customer_master")
    print("Write success")
except Exception as e:
    print(f"Write failed : {e}")

25/06/12 08:24:21 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


Write success


In [ ]:

# Load target as DeltaTable
delta_target = DeltaTable.forPath(spark, "../DeltaLake/customer_master")

'''Perform Merge/Upsert Operation'''
delta_target.alias("tgt").merge(
    customer_master_src.alias("src"), "tgt.customer_id == src.customer_id"
).whenMatchedUpdate(set={
    'full_name':'src.full_name',
    'email': 'src.email',
    'address':'src.address',
    'phone':'src.phone',
    'updated_at':current_timestamp(),
    'current_flag':"'Y'"
}).whenNotMatchedInsert(values={
    'customer_id':'src.customer_id',
    'full_name':'src.full_name',
    'email': 'src.email',
    'address':'src.address',
    'phone':'src.phone',
    'created_at':'src.created_at',
    'updated_at':current_timestamp(),
    'current_flag':"'Y'"
}).execute()

tgt=spark.read.format("delta").load("../DeltaLake/customer_master")
tgt.show(truncate=False)


+-----------+----------+--------------------+------------+----------+--------------------+--------------------+------------+
|customer_id| full_name|               email|     address|     phone|          created_at|          updated_at|current_flag|
+-----------+----------+--------------------+------------+----------+--------------------+--------------------+------------+
|          1|  John Doe|john.doe@example.com|123 Maple St|1234567890|2025-06-12 08:24:...|2025-06-12 08:31:...|           Y|
|          2|Jane Smith|jane.smith@exampl...| 456 Oak Ave|9876543210|2025-06-12 08:24:...|2025-06-12 08:31:...|           Y|
|          3| David Lee|david.lee@example...| 789 Pine Rd|6300500200|2025-06-12 08:24:...|2025-06-12 08:31:...|           Y|
|          4|Harvey Lee|davidson.lee@exam...| 700 Pine Rd|7730061615|2025-06-12 08:24:...|2025-06-12 08:31:...|           Y|
+-----------+----------+--------------------+------------+----------+--------------------+--------------------+------------+


In [ ]:
'''Perform SCD Type 2 on it'''



In [ ]:
# Time Travel (Query Past Versions)

df_old = spark.read.format("delta").option("versionAsOf", 0).load("delta_tables/customer")
df_old.show()

In [ ]:
 Schema Enforcement and Evolution

    Enforcement: Will error out if new data doesn’t match schema.

    Evolution: You can enable schema evolution when appending.

df_new.write.format("delta") \
    .option("mergeSchema", "true") \
    .mode("append").save("delta_tables/customer")

In [2]:
tgt=spark.read.format("delta").load("../DeltaLake/numbers")
tgt.show(truncate=False)

25/06/14 23:44:05 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------------+-----------------------+
|message            |batch_time             |
+-------------------+-----------------------+
|Current Number is 0|2025-06-14 23:31:11.351|
|Current Number is 1|2025-06-14 23:31:11.351|
|Current Number is 2|2025-06-14 23:31:11.351|
|Current Number is 3|2025-06-14 23:31:11.351|
|Current Number is 4|2025-06-14 23:31:11.351|
|Current Number is 5|2025-06-14 23:31:11.351|
|Current Number is 6|2025-06-14 23:31:11.351|
|Current Number is 7|2025-06-14 23:31:11.351|
|Current Number is 8|2025-06-14 23:31:11.351|
|Current Number is 9|2025-06-14 23:31:11.351|
|Current Number is 0|2025-06-14 23:30:09.1  |
|Current Number is 1|2025-06-14 23:30:09.1  |
|Current Number is 2|2025-06-14 23:30:09.1  |
|Current Number is 3|2025-06-14 23:30:09.1  |
|Current Number is 4|2025-06-14 23:30:09.1  |
|Current Number is 5|2025-06-14 23:30:09.1  |
|Current Number is 6|2025-06-14 23:30:09.1  |
|Current Number is 7|2025-06-14 23:30:09.1  |
|Current Number is 8|2025-06-14 23